<a href="https://colab.research.google.com/github/hydrogradCreator/2022-1-exercicio-revisao-refatoracao/blob/main/ListaAM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lista Aberta do Módulo 4 - Comparando Classificadores

**Regras para envio:**

1. Faça uma cópia deste arquivo
2. Deixe o arquivo aberto para qualquer pessoa com o link (Share > Anyone with the link)
3. Garanta que as saídas do notebook estão salvas (Settings > deixe desligada a opção (New notebooks use private outputs (omit outputs when saving)), e acesse o link depois em outro navegador para garantir que estão!)
4. Digite seu nome ##AQUI###
5. Após finalizar o exercício, poste o link (apenas o link!) no moodle.

**Atenção**: como temos pouco tempo para o fechamento do semestre, por favor garantam que seus links estão corretos! Não teremos tempo para correções. É essencial que os alunos de ICD saibam usar o Google Colab.

## Identificação de malignidade em tumores de mama - UCI ML Breast Cancer Wisconsin (Diagnostic)

Neste trabalho você deverá realizar uma comparação entre o k-Nearest Neighbors (KNN) e o Random Forest para classificação de pacientes com tumores na mama (maligno = 1 vs benigno = 0). Para isso, usaremos o dataset de câncer de mama de UCI (já embutido no sklearn).

A avaliação dos métodos deverá ser feito usando precisão, revocação e F1. Você também deve reportar a matriz de confusão.

Você não precisa implementar os métodos, já que estão disponíveis na biblioteca scikit-learn da linguagem Python. Se necessário, pode fazer mais importações de bibliotecas.

## Importando módulos e baixando dataset

In [14]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


In [2]:
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data,columns =[cancer.feature_names])

In [3]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
#Para acessar as labels
cancer.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [5]:
#Pegar as colunas que serão usadas na classificação
features = df.columns

# Número de folds da validação cruzada
SPLITS = 5

Se tiver dúvidas sobre a validação cruzada, recomendo :https://towardsdatascience.com/what-is-stratified-cross-validation-in-machine-learning-8844f3e7ae8e . Para regressão, vocês usaram o kfold, agora recomendo a utilização do StratifiedKFold.

# Exercício 01:

Use o RandomForest para classificar as pacientes com e sem câncer.

Para a questão, faça as seguintes tarefas:
* Varie a o número de árvores (n_estimators) e a profundidade máxima da árvore (n_estimators). Faça pelo menos 4 configurações.
* Realize validação cruzada, com k-fold igual a 5
* Reporte a precisão, revocação, F1 e a matriz de confusão para cada configuração


In [9]:
# YOUR CODE HERE
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
X = df.values
y = cancer.target

# Definir as configurações de n_estimators e max_depth
configurations = [(10, 5), (10, 10), (50, 5), (50, 10)]

# Número de folds da validação cruzada
SPLITS = 5
kf = KFold(n_splits=SPLITS)

for n_estimators, max_depth in configurations:
    print(f"Configuração: n_estimators={n_estimators}, max_depth={max_depth}")
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    # Realizar validação cruzada
    y_pred = cross_val_predict(model, X, y, cv=kf)

    # Calcular e imprimir as métricas
    accuracy = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Confusion Matrix:\n{conf_matrix}\n")


Configuração: n_estimators=10, max_depth=5
Accuracy: 0.9490333919156415
Recall: 0.969187675070028
F1 Score: 0.9597780859916782
Confusion Matrix:
[[194  18]
 [ 11 346]]

Configuração: n_estimators=10, max_depth=10
Accuracy: 0.945518453427065
Recall: 0.9607843137254902
F1 Score: 0.9567642956764296
Confusion Matrix:
[[195  17]
 [ 14 343]]

Configuração: n_estimators=50, max_depth=5
Accuracy: 0.9595782073813708
Recall: 0.9775910364145658
F1 Score: 0.9680998613037448
Confusion Matrix:
[[197  15]
 [  8 349]]

Configuração: n_estimators=50, max_depth=10
Accuracy: 0.9543057996485061
Recall: 0.9663865546218487
F1 Score: 0.9636871508379888
Confusion Matrix:
[[198  14]
 [ 12 345]]



# Exercício 02:

Use a knn para classificar as pacientes com e sem câncer.

Para a questão, faça as seguintes tarefas:

* Escolha e reporte resultados com 3 números de vizinhos diferentes (k)
* Realize validação cruzada, com k-fold igual a 5 (para cada um dos k vizinhos)
* Reporte a precisão, revocação, F1 e a matriz de confusão para cada configuração


In [10]:
# Definir os valores de k para kNN
k_values = [3, 5, 7]

# Número de folds da validação cruzada
SPLITS = 5
kf = KFold(n_splits=SPLITS)

# Avaliar cada valor de k
for k in k_values:
    print(f"Configuração: k={k}")
    model = KNeighborsClassifier(n_neighbors=k)

    # Realizar validação cruzada
    y_pred = cross_val_predict(model, X, y, cv=kf)

    # Calcular e imprimir as métricas
    accuracy = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Confusion Matrix:\n{conf_matrix}\n")

Configuração: k=3
Accuracy: 0.9209138840070299
Recall: 0.9551820728291317
F1 Score: 0.938101788170564
Confusion Matrix:
[[183  29]
 [ 16 341]]

Configuração: k=5
Accuracy: 0.9261862917398945
Recall: 0.9635854341736695
F1 Score: 0.9424657534246577
Confusion Matrix:
[[183  29]
 [ 13 344]]

Configuração: k=7
Accuracy: 0.9226713532513181
Recall: 0.957983193277311
F1 Score: 0.9395604395604396
Confusion Matrix:
[[183  29]
 [ 15 342]]



# Exercício 03:

Compare os resultados (F1 de cada método com cada configuração) em um gráfico. Qual foi o melhor método? Houve diferença significativa com as variações de parâmetros?

As configurações do RandomForest geralmente alcançaram F1 Scores mais altos em comparação com as configurações do kNN.
A configuração com o melhor desempenho foi o RandomForest com n_estimators=50 e max_depth=5, alcançando o F1 Score mais alto entre todas as configurações testadas.
As variações nos parâmetros n_estimators e max_depth para RandomForest mostraram uma diferença no desempenho, mas não extremamente significativa.




 O aumento no número de estimadores e a profundidade máxima tendeu a melhorar o desempenho até certo ponto.
Para o kNN, variações no número de vizinhos (k) também resultaram em diferenças no desempenho, mas estas foram relativamente menores.
O melhor desempenho foi observado com k=5.
Concluindo, o método RandomForest com mais estimadores e maior profundidade apresentou um desempenho melhor em termos de F1 Score para este conjunto de dados.

# Exercício 04:

Escolha a melhor configuração/método e refaça o modelo, agora usando apenas 5 features. (Você pode selecionar aleatoriamente ou usar alguma metodologia de feature selection). O que aconteceu com a F1? Por quê?

In [17]:
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
model.fit(X_train, y_train)

feature_importances = pd.Series(model.feature_importances_, index=X.columns).nlargest(5)
selected_features = feature_importances.index

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

model.fit(X_train_selected, y_train)
y_pred_selected = model.predict(X_test_selected)

new_f1_score = f1_score(y_test, y_pred_selected)
print("F1 Score com 5 features selecionadas:", new_f1_score)

F1 Score com 5 features selecionadas: 0.962962962962963


significa que o modelo com menos features teve um desempenho quase tão bom quanto o modelo completo. o que sugere que as features selecionadas eram altamente representativas e informativas para a tarefa de classificação, isso também indica que, para este conjunto de dados específico, um modelo mais simples (com menos features) pode ser suficiente para alcançar um desempenho de classificação quase tão bom quanto um modelo mais complexo.